# Self supervised Learning

Pretrain a "foundation model" using data without outcomes, from a large unlabeled dataset. Note that the idea behind the pre-training is to guide the fine tunning task (survival prediction). **Note that given this is a toy example the pre-training effect may be marginal.** 

The input to the mode are the features from the dataset and **not the outcomes**

**Note that this notebook may take a while to complete when number of epochs is large**: For self supervised learning it is recommended to use a large  number of epochs. For illustration, we ran it for 1,000 epochs. 

In [1]:
import os
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/home/shenwanxiang/anaconda3"
import tensorflow as tf
tf.test.is_gpu_available()
tf.__version__

2025-08-12 19:03:52.743305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-12 19:03:52.799583: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-12 19:03:53.147957: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/shenwanxiang/anaconda3/lib:
2025-08-12 19:03:53.147990: W tensorflow/compiler/xla/stream_exe

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2025-08-12 19:03:53.557343: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-12 19:03:53.674002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-12 19:03:53.674108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-12 19:03:53.690109: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at

'2.11.0'

In [2]:
import sys
sys.path.append('../codeocean/environment/clinical_transformer/')

import pandas as pd
from collections import Counter

from xai.models.SimplifiedClinicalTransformer.Trainer import Trainer
from xai.losses.survival import cIndex_SigmoidApprox as cindex_loss
from xai.metrics.survival import sigmoid_concordance as cindex

from xai.models import Trainer
from xai.models import SelfSupervisedTransformer
from xai.models import OptimizedSelfSupervisedDataGenerator as SelfSupervisedDataGenerator

from xai.losses.selfsupervision.classifier_regression import CompositeLoss

In [3]:
from samecode.random import set_seed

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Parameters

In [5]:
max_features_percentile=100
test_size=0.1
mode='self-supervision'
learning_rate=0.0001
repetitions=1
epochs=2000
verbose=2
seed=0
embedding_size = 128
num_heads = 2
num_layers = 2

loss = CompositeLoss(feature_w=1, value_w=0.1) # Contribution of individual losses (predicts keys, values) 

In [6]:
data = pd.read_csv('./data/dataset-pretrain.data.csv')
features = data.columns[-29:].tolist()
#data = data[features]

In [7]:
!rm -r ./results/runs/FoundationModel

/bin/sh: /home/shenwanxiang/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/sh)


In [ ]:
set_seed(0)
outdir = './results/runs/FoundationModel/'

trainer = Trainer(
    out_dir = outdir,
    max_features_percentile=max_features_percentile,
    test_size=test_size,
    mode=mode,
    model=SelfSupervisedTransformer, 
    dataloader=SelfSupervisedDataGenerator,
    loss=loss,
    metrics=[]
)

trainer.setup_data(
    data, 
    discrete_features = [],
    continuous_features = features,
)

trainer.setup_model(
    embedding_size=embedding_size, 
    num_heads=num_heads, 
    num_layers=num_layers,
    learning_rate=learning_rate,
    batch_size_max = True,
    batch_size=10000, # This will take a batch with the size of the training / testing shape,
    save_best_only=False
)

trainer.fit(repetitions=repetitions, epochs=epochs, verbose=verbose, seed=seed)

/home/shenwanxiang/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO	2025-08-12 19:03:55,466	Setting up working directory: ./results/runs/FoundationModel/
INFO	2025-08-12 19:03:55,467	Number of continuous features: 29
INFO	2025-08-12 19:03:55,468	Number of discrete features: 0
INFO	2025-08-12 19:03:55,468	Number of samples: 10184
INFO	2025-08-12 19:03:55,478	Number of classes: 37
INFO	2025-08-12 19:03:55,478	RUN ID: fold-0_id-0
INFO	2025-08-12 19:03:55,479	RUN ID out directory: ./results/runs/FoundationModel//fold-0_id-0/
INFO	2025-08-12 19:03:56,130	Training samples: 9165
INFO	2025-08-12 19:03:56,131	Testing samples: 1019
INFO	2025-08-12 19:03:56,148	Number of features at 100th percentile: 29 that are non nans
2025-08-12 19:03:56.149381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_

Epoch 1/2000
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2025-08-12 19:03:56.814223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
WARNING	2025-08-12 19:03:56,954	AutoGraph could not transform <function Model.make_train_function.<locals>.train_function at 0x7f9d5a44b9e0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


WARNING	2025-08-12 19:03:56,960	AutoGraph could not transform <bound method SelfSupervisedTransformer.call of <xai.models.SimplifiedClinicalTransformer.Topologies.BertLikeTransformer.BertLikeAttention.SelfSupervisedTransformer object at 0x7f9d6a666390>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2025-08-12 19:03:58.788800: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f9b4003d320 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-12 19:03:58.788865: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2025-08-12 19:03:58.788878: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2025-08-12 19:03:58.799440: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-12 19:03:58.838803: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-08-12 19:03:58.858175: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


WARNING	2025-08-12 19:04:00,332	AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f9c12527ef0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 - 4s - loss: 0.9035 - val_loss: 0.8550 - 4s/epoch - 4s/step
Epoch 2/2000
1/1 - 0s - loss: 0.8872 - val_loss: 0.8372 - 342ms/epoch - 342ms/step
Epoch 3/2000
1/1 - 0s - loss: 0.8678 - val_loss: 0.8145 - 335ms/epoch - 335ms/step
Epoch 4/2000
1/1 - 0s - loss: 0.8511 - val_loss: 0.7951 - 337ms/epoch - 337ms/step
Epoch 5/2000
1/1 - 0s - loss: 0.8373 - val_loss: 0.7758 - 374ms/epoch - 374ms/step
Epoch 6/2000
1/1 - 0s - loss: 0.8236 - val_loss: 0.7571 - 335ms/epoch - 335ms/step
Epoch 7/2000
1/1 - 0s - loss: 0.8091 - val_loss: 0.7405 - 339ms/epoch - 339ms/step
Epoch 8/2000
1/1 - 0s - loss: 0.7989 - val_loss: 0.7269 - 339ms/epoch - 339ms/step
Epoch 9/2000
1/1 - 0s - loss: 0.7894 - val_l

In [ ]:
from xai.models import clean_run

clean_run(
    path='./results/runs/FoundationModel/',
    keep=[1, 10, 100, 500, 1000, 1500, 2000]
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

dfh = pd.read_csv("./results/runs/FoundationModel//fold-0_id-0/history.csv", index_col=0)
fig, ax=plt.subplots(figsize=(5,4))
dfh.plot(ax=ax)
ax.set_ylabel('SSL loss')